# Model Context Protocol (MCP) Implementation with OpenAI Agents SDK

This notebook demonstrates how to configure and run an **OpenAI Agent** using multiple **Model Context Protocol (MCP)** servers to enable specialized tool usage.

### Node and Playwright Dependencies
The **Playwright/Browser** and **Filesystem** MCP servers require Node.js and the **npx** command. Ensure Node.js and Playwright are installed, and that `npx` is available in your system's PATH.

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import asyncio 

In [2]:
load_dotenv(override=True)

SANDBOX_PATH = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
if not os.path.exists(SANDBOX_PATH):
    os.makedirs(SANDBOX_PATH)


---

## MCP Server Configuration and Tool Collection

MCP servers are configured for specific functionalities. This aligns with the **Single Responsibility Principle (SRP)**, where each server is dedicated to a single domain (e.g., Fetch, Browser, Filesystem).

In [3]:
# MCP Server Configurations
FETCH_CONFIG = {"command": "uvx", "args": ["mcp-server-fetch"]}
PLAYWRIGHT_CONFIG = {"command": "npx", "args": ["@playwright/mcp@latest"]}
FILES_CONFIG = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", SANDBOX_PATH]}

async def get_mcp_tools(config):
    """
    Initializes an MCP server with the given config and lists its available tools
    """
    try:
        # Note: Depending on the SDK version, server.list_tools() or server.session.list_tools() may be required.
        async with MCPServerStdio(params=config, client_session_timeout_seconds=60) as server:
            tools = await server.list_tools()
            return tools
    except Exception as e:
        print(f"Error: Failed to start or retrieve tools from {config['args'][0]} server. Details: {e}")
        return None

# Collect all necessary tools
async def collect_all_tools():
    print("Retrieving Fetch tools...")
    fetch_tools = await get_mcp_tools(FETCH_CONFIG)
    print("Retrieving Playwright (Browser) tools...")
    playwright_tools = await get_mcp_tools(PLAYWRIGHT_CONFIG)
    print("Retrieving Filesystem tools...")
    file_tools = await get_mcp_tools(FILES_CONFIG)
    
    # Return all tool results, handling failures gracefully later
    return fetch_tools, playwright_tools, file_tools

try:
    all_tools = await collect_all_tools() 
    fetch_tools, playwright_tools, file_tools = all_tools
except NameError:
    import nest_asyncio
    nest_asyncio.apply()
    all_tools = asyncio.run(collect_all_tools())
    fetch_tools, playwright_tools, file_tools = all_tools

print("\n--- Tool Summary ---")
print(f"Fetch Tools: {fetch_tools[0].name if fetch_tools else 'None'}")
print(f"Playwright Tools: {playwright_tools[0].name if playwright_tools else 'None'}")
print(f"Filesystem Tools: {file_tools[0].name if file_tools else 'None'}")


Retrieving Fetch tools...
Retrieving Playwright (Browser) tools...
Retrieving Filesystem tools...

--- Tool Summary ---
Fetch Tools: fetch
Playwright Tools: browser_close
Filesystem Tools: read_file


---

## Multi-Tool Integration and Agent Execution

The filesystem and browser tools are injected into the agent, adhering to the **Dependency Inversion Principle (DIP)** where the high-level module (the Agent) depends on abstractions (the MCP Servers/Tools).

In [4]:
AGENT_INSTRUCTIONS = """
You browse the internet and use the file system to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""

async def run_agent_task(files_config, playwright_config, instructions, task):
    """
    Starts the agent with required MCP servers and executes the task.
    """
    try:
        # Use nested async context managers to ensure both servers are properly started and stopped.
        async with MCPServerStdio(params=files_config, client_session_timeout_seconds=60) as mcp_server_files:
            async with MCPServerStdio(params=playwright_config, client_session_timeout_seconds=60) as mcp_server_browser:
                # Agent Initialization
                agent = Agent(
                    name="investigator", 
                    instructions=instructions, 
                    model="gpt-4.1-mini",
                    mcp_servers=[mcp_server_files, mcp_server_browser]
                )
                
                with trace("investigate"):
                    print(f"Agent starting task: {task}")
                    result = await Runner.run(agent, task, max_turns=20)
                    return result.final_output
    except Exception as e:
        return f"Critical error during agent execution: {e}"


TASK = "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md"

try:
    final_output = await run_agent_task(FILES_CONFIG, PLAYWRIGHT_CONFIG, AGENT_INSTRUCTIONS, TASK) 
    print("\n--- Final Output ---")
    print(final_output)
except NameError:
    final_output = asyncio.run(run_agent_task(FILES_CONFIG, PLAYWRIGHT_CONFIG, AGENT_INSTRUCTIONS, TASK))
    print("\n--- Final Output ---")
    print(final_output)


Agent starting task: Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md

--- Final Output ---
I found a wonderful Banoffee Pie recipe from BBC Good Food. I will provide you the summarized recipe in markdown format here for your convenience:

```markdown
# Banoffee Pie Recipe

An easy family favorite with buttery pastry and sweet dulce de leche, topped with cream and optional dark chocolate garnish.

## Ingredients

### For the filling:
- 4 bananas, sliced
- 394g caramel or dulce de leche
- 300ml double cream
- Dark chocolate (optional, for garnish)

### For the pastry:
- 100g butter, chilled (plus extra for greasing)
- 200g plain flour
- 1 medium egg, separated
- 1 tbsp golden caster sugar

## Method

1. Make the pastry: pulse butter and flour in a food processor until breadcrumbs form. Add egg yolk and sugar, pulse again, then add cold water gradually until dough forms.
2. Knead dough gently, wrap in cling film, chill for 30 mins.
3. Heat oven to 190C/1

---

## Resources and Further Exploration

Review the agent's step-by-step trace:
[OpenAI Traces Platform](https://platform.openai.com/traces)

MCP marketplaces and community resources:
* [mcp.so](https://mcp.so)
* [glama.ai/mcp](https://glama.ai/mcp)
* [smithery.ai/](https://smithery.ai/)
* [HuggingFace - Top 11 MCP Libraries](https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries)
* [HuggingFace - MCP Community Article](https://huggingface.co/blog/Kseniase/mcp)